In [0]:
!wget -q https://l1nna.com/372/Assignment/A2-3/train.csv
!wget -q https://l1nna.com/372/Assignment/A2-3/test.csv

In [0]:
import pandas as pd
import csv


xy_train_df = pd.read_csv('train.csv')
x_test_df  = pd.read_csv('test.csv', index_col='id')


xy_train_df['length'] = xy_train_df.apply(lambda x: len(x.review), axis=1)
xy_train_df = xy_train_df.sort_values('length')
xy_train_df

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

vocab_size = 10000
max_len = 256

xy_train, xy_validation = train_test_split(xy_train_df, test_size=0.2)

# build vocabulary from training set
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(xy_train.review)

# padding is done inside: 
x_train = tokenizer.texts_to_matrix(xy_train.review, mode='binary')[:, :max_len]
y_train = xy_train.rating

x_valid = tokenizer.texts_to_matrix(xy_validation.review, mode='binary')[:, :max_len]
y_valid = xy_validation.rating

x_test = tokenizer.texts_to_matrix(x_test_df.review, mode='binary')[:, :max_len]

print(x_train.shape)
print(x_valid.shape)
print(x_test.shape)

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import collections
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras


import tensorflow as tf
from tensorflow.keras.optimizers import Adam

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 20))
model.add(keras.layers.CuDNNGRU(100))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))


model.compile(
    optimizer=Adam(clipnorm=4.),
    loss='binary_crossentropy',
    metrics=['accuracy'])



history = model.fit(x_train,
                    y_train,
                    epochs=15,
                    batch_size=64,
                    validation_data=(x_valid, y_valid),
                    verbose=1)

In [0]:
model.evaluate(x_valid, y_valid)

In [0]:
y_predict = np.squeeze(model.predict_classes(x_valid))

from sklearn.metrics import  f1_score
from sklearn.metrics import confusion_matrix

print(f1_score(y_valid, y_predict, average='micro'))

In [0]:
# run on testing set:
y_predict = np.squeeze(model.predict_classes(x_test))

pd.DataFrame(
    {'id': x_test_df.index,
     'rating':y_predict}).to_csv('sample_submission.csv', index=False)